In [178]:
import os
# Change to your own
PATH_NAME = 'C:/Users/woobi/Documents/LearningFoodPreferences2/data/'


In [179]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\woobi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [180]:
import pandas as pd
import os
from nltk.stem import WordNetLemmatizer
import numpy as np
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
import pickle
import math
import re
from collections import Counter

In [181]:
# Load word2vec model
model = KeyedVectors.load(PATH_NAME + "model_added_sentences.kv", mmap='r')

In [182]:
# Getting a sample of what an embedding looks like
model["milk"]

array([-0.18189265,  0.06970767,  0.31639957, -0.01767082,  0.5187955 ,
        0.5602029 ,  0.9045883 , -1.054123  , -0.01921861,  0.36951056,
        0.3355017 , -0.12449828, -0.30174094,  0.5924666 , -0.51961124,
        0.31225696, -0.33364275, -0.18640412,  0.07013486, -0.23054427,
        0.09309182,  0.15374434, -0.07223768,  0.06692434, -0.2732636 ,
        0.19992387, -0.29497775,  0.09157794, -0.18872151, -0.297041  ,
        0.32707772, -0.61746264,  0.62452805,  0.45915222, -0.95687073,
        0.4706721 ,  0.39204383, -0.43490675,  0.11465851,  1.0464789 ,
       -0.02223825,  0.26908487, -0.4449049 , -0.6961393 , -0.9325207 ,
        0.37337124, -0.12486531,  0.629239  , -0.58858454, -0.5264343 ,
       -0.22465397, -0.27662367, -0.7574381 ,  0.13795258, -0.28194296,
       -0.4998588 , -0.4895984 , -0.02067281,  0.10521271, -0.34428862,
        0.31613484,  0.02247572, -0.6983412 ,  0.7814676 ,  0.74939597,
        0.21136394, -0.4546939 ,  0.81796235, -0.09250929,  0.19

In [183]:
# Load WWEIA food categories csv. Contains the food category codes and associated verbal food categories.
# The "same_category" column lists which categories are considered synonymous.
wweia_food_categories = pd.read_csv(PATH_NAME + 'wweia_food_categories_addtl.csv')
wweia_food_categories

,NO,식품명,식품대분류,식품상세분류
0,1,꿩불고기,구이류,육류구이
1,2,닭갈비,구이류,육류구이
2,3,닭갈비,구이류,육류구이
3,4,닭꼬치,구이류,육류구이
4,5,더덕구이,구이류,채소류구이
...,...,...,...,...
7678,7679,<아내의 식탁>짜.수.치,볶음류,떡볶이류
7679,7680,<아내의 식탁>바질베스토감바스,볶음류,어패류볶음
7680,7681,<아내의 식탁>밀푀유스키야키,찌개 및 전골류,채소류찌개.전골
7681,7682,<파빌리온>양갈비 스테이크,구이류,육류구이


In [184]:
# Load the vocabulary of WWEIA food descriptions

food_words = pd.read_csv(PATH_NAME + 'food_words.csv')
food_words = food_words.iloc[:,1]
food_words = food_words.to_list()
food_words

['꿩불고기',
 '닭갈비',
 '닭갈비',
 '닭꼬치',
 '더덕구이',
 '돼지갈비',
 '불고기',
 '붕장어소금구이',
 '소양념갈비구이',
 '양념왕갈비',
 '양념장어구이',
 '임연수구이',
 '햄버그스테이크',
 '황태구이',
 '훈제오리',
 '게국지',
 '곰치국',
 '굴국밥',
 '김치국',
 '떡만둣국',
 '만둣국',
 '맑은감자국',
 '매생이국',
 '무된장국',
 '미소된장국',
 '바지락조개국',
 '뼈다귀해장국',
 '선지해장국',
 '소고기무국',
 '시래기된장국',
 '올갱이국',
 '우거지해장국',
 '우렁된장국',
 '홍합미역국',
 '황태해장국',
 '고추잡채',
 '난자완스',
 '도토리묵말이',
 '라조기',
 '물회',
 '생선물회',
 '소고기샤브샤브',
 '오징어순대',
 '초당순두부',
 '해파리냉채',
 '고들빼기',
 '열무얼갈이김치',
 '미나리나물',
 '취나물',
 '경단(깨)',
 '경단(콩)',
 '기피편',
 '녹두시루떡',
 '메밀전병',
 '무지개떡',
 '송편(깨)',
 '송편(콩)',
 '수수부꾸미',
 '수수팥떡',
 '시루떡',
 '쑥떡',
 '절편',
 '찹쌀떡',
 '간자장',
 '굴짬뽕',
 '기스면',
 '김치라면',
 '김치우동',
 '냉김치말이국수',
 '닭칼국수',
 '둘깨칼국수',
 '막국수',
 '삼선우동',
 '삼선자장면',
 '삼선짬뽕',
 '쌀국수',
 '열무김치국수',
 '열무냉면',
 '오일소스스파게티',
 '올갱이국수',
 '우동(일식)',
 '우동(중식)',
 '울면',
 '자장면',
 '짬뽕라면',
 '치즈라면',
 '크림소스스파게티',
 '토마토소스스파게티',
 '해물칼국수',
 '해물크림소스스파게티',
 '해물토마토소스스파게티',
 '골뱅이무침',
 '노각무침',
 '단무지무침',
 '달래나물무침',
 '더덕무침',
 '도토리묵',
 '밴댕이무침',
 '서대회무침',
 '쑥갓나물무침',
 '쥐치채',
 '청포묵무침',
 '탕평채',
 '홍어무침',


In [185]:
# Load wweia_data, which contains the FNDDS foods, their nutritional information, and their food category

wweia_data = pd.read_csv(PATH_NAME + 'wweia_data.csv')
wweia_data

,식품코드,식품명,NO,에너지(㎉),수분(g),단백질(g),지방(g),탄수화물(g),총당류(g),자당(g),...,에이코사트리에노산(20:3(n-6))(g),에이코사펜타에노산(20:5(n-3))(g),도코사펜타에노산(22:5(n-3))(g),도코사헥사에노산(22:6(n-3))(g),트랜스 지방산(g),트랜스 올레산(18:1(n-9)t)(g),트랜스 리놀레산(18:2t)(g),트랜스 리놀렌산(18:3t)(g),회분(g),카페인(㎎)
0,D000006,꿩불고기,1,368.8,412.6,33.5,8.5,39.7,16.9,7.2,...,0.14,0,0,0.02,0.1,0.02,0.05,0,5.8,-
1,D000007,닭갈비,2,595.61,276.4,45.9,25.8,44.9,21.2,3.6,...,0.13,0.01,0.02,0.01,0.2,0.08,0.06,0.01,7,-
2,D000008,닭갈비,3,558.47,195.4,45.5,31.6,23.1,8.5,0.6,...,0.16,0.01,0.02,0.02,0.3,0.14,0.08,0.03,4.5,-
3,D000009,닭꼬치,4,176.72,35.31,11.56,8.57,13.35,3.15,0.11,...,0,0,0,0,0.04,0,0,0,1.22,-
4,D000010,더덕구이,5,184,58.1,3.1,5.2,31.1,11.6,2.2,...,0,0,0,0,0,0,0,0,2.5,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7678,D018919,<아내의 식탁>짜.수.치,7679,251,-,9,7,38,9,-,...,-,-,-,-,0.1,-,-,-,-,-
7679,D018920,<아내의 식탁>바질베스토감바스,7680,324,-,7,28,11,0,-,...,-,-,-,-,0.1,-,-,-,-,-
7680,D018921,<아내의 식탁>밀푀유스키야키,7681,110,-,8,6,6,4,-,...,-,-,-,-,0.4,-,-,-,-,-
7681,D018922,<파빌리온>양갈비 스테이크,7682,177,-,10,13,5,4,-,...,-,-,-,-,0.5,-,-,-,-,-


In [186]:
# Load embeddings
wweia_embeddings = np.loadtxt(PATH_NAME + 'wweia_embeddings_added_sen.csv', delimiter = ",")
wweia_embeddings.shape

(7682, 300)

In [187]:
def get_most_pop_categories(curr_log, n, col_name):

  cats = curr_log.loc[:, col_name]

  cats_counter = Counter(cats)
  counter_results = cats_counter.most_common(n)
  
  counter_list = [elem[0] for elem in counter_results]
  
  return counter_list

In [188]:
# initialize lemmatizer
lemmatizer = WordNetLemmatizer()

In [189]:
unhelpful_words = ["with", "or", "cooked", "to", "and", "as", "ns", "cooking", "in", "made", "from", " ", "canned", "frozen", "type", "eaten", "of", "on", "fresh", "nfs", "ready", "strained", "style", "than", "prepared", "method", "stewed", "drained", "homemade", "home"]

In [190]:
# Only keep words in phrase that are in food_words

def reduce_with_food_words(comma_phrase):
  comma_phrase_reduced = [lemmatizer.lemmatize(word.lower()) for word in comma_phrase.split() if (lemmatizer.lemmatize(word.lower()) in food_words or word.lower() in food_words) and (lemmatizer.lemmatize(word.lower()) not in unhelpful_words or word.lower() not in unhelpful_words)]
  return comma_phrase_reduced

In [191]:
# Format of food names seems to be (Optional) Company, Food, Details about food
# run function to guess if first or second comma phrase has Food

def process_food_log(curr_log, wweia_synonym_cats):
  curr_log['predicted_categories_number'] = 0
  curr_log['predicted_categories_words'] = ""
  curr_log['max_cosim_score'] = 0
  curr_log['most_sim_food'] = ""
  curr_log['reciprocal_rank'] = 0.0
  curr_log['sym_reciprocal_rank'] = 0.0

  for i in range(curr_log.shape[0]):
    descrip = curr_log.loc[i, 'Food Name']
    descrip_split = descrip.split(", ")
    print(descrip_split)
    # Reduce food log description to words in food_words
    first_phrase_num_words = reduce_with_food_words(descrip_split[0])
    pre_embedding = []

    if len(descrip_split) > 1:
      second_phrase_num_words = reduce_with_food_words(descrip_split[1])
      descrip_split_0_words = descrip_split[0].split()
      descrip_split_1_words = descrip_split[1].split()

      begin_first = False if len(second_phrase_num_words)/len(descrip_split_1_words) > len(first_phrase_num_words)/len(descrip_split_0_words) else True
      
      if begin_first:
        last_phrase = descrip

      if begin_first == False:
        three_phrases = descrip.partition(", ")
        last_phrase = three_phrases[2]

    else:
      last_phrase = descrip
    last_phrase_comma = last_phrase.split(", ")
    for j in range(len(last_phrase_comma)):
      split_comma = last_phrase_comma[j].split(" ")
      for elem in split_comma:
        elem = elem.lower()
        if elem not in unhelpful_words:
          pre_embedding.append(elem)

    word_embed = np.zeros(shape = (1, len(model["sushi"])))
    if len(pre_embedding) > 0:
      # Turned edited_descrip into embedding vector by averaging the words
      word_embed = np.zeros(shape = (1, len(model["sushi"])))
      num_words = 0
      for word in pre_embedding:
        word = word.lower()
        if word in model:
          num_words += 1
          word_embed += model[word]

      if num_words != 0:
        word_embed /= num_words
    # Compare to the other vectors
    similarities = cosine_similarity(word_embed, wweia_embeddings) # (1, 7918) for each wweia food
    # Finding WWEIA foods with highest similarity with each
    # food log food. Row has sorted list of WWEIA food IDs 
    # for each food log food.
    to_keep = np.sort(similarities, axis=1)
    to_keep_args = np.argsort(similarities, axis=1)
    indices = np.flip(to_keep_args, axis = 1)
    
    sym_rank = 1000000
    # record RR here
    for index in range(indices.shape[1]):
      true_cat = curr_log.loc[i, 'wweia_food_category_code']
      if math.isnan(true_cat): continue
      if wweia_data.loc[indices[0,index], 'NO'] == true_cat:
        rank = index
        rr = 1 / (rank+1)
        if sym_rank > rank:
          sym_rank = rank
          sym_rr = 1 / (rank+1)

        break
      else:
        
      #   if wweia_data.loc[indices[0,index], 'NO'] in wweia_synonym_cats[true_cat]:
        if sym_rank > index:
          sym_rank = index
          sym_rr = 1 / (index+1)
    most_sim_food_index = indices[0,0]

    most_sim_food_row = wweia_data.iloc[most_sim_food_index, :]
    highest_cat_num = most_sim_food_row['NO']
    print(highest_cat_num)
    highest_cat_words = wweia_food_categories.loc[wweia_food_categories['NO'] == highest_cat_num, '식품명']
    curr_log.loc[i, 'predicted_categories_number'] = highest_cat_num
    curr_log.loc[i, 'predicted_categories_words'] = highest_cat_words.to_list()[0]
    curr_log.loc[i, 'max_cosim_score'] = np.array2string(to_keep[0,-5:])
    curr_log.loc[i, 'most_sim_food'] = most_sim_food_row['식품명']
    curr_log.loc[i, 'reciprocal_rank'] = rr
    curr_log.loc[i, 'sym_reciprocal_rank'] = sym_rr
  return curr_log

In [192]:
def get_metrics(curr_log, wweia_synonym_cats):

  # Will hold foods already seen so foods aren't double-counted
  # when calculating accuracy
  seen_foods = set()
  total_valid_foods = 0
  num_corr = 0
  num_sym = 0

  for i in range(curr_log.shape[0]):
    
    true_cat = curr_log.loc[i, 'wweia_food_category_code']
    false_cat = curr_log.loc[i, 'predicted_categories_number']

    prev_set_size = len(seen_foods)
    seen_foods.add(curr_log.loc[i, 'Food Name'])

    if math.isnan(true_cat): continue # food does not have valid category, i.e. antacid

    if len(seen_foods) > prev_set_size: # not a repeated food

      if true_cat == false_cat:
        num_corr += 1
        num_sym += 1

      else:
      #   if false_cat in wweia_synonym_cats[true_cat]:
        num_sym += 1
        
      total_valid_foods += 1

  acc = num_corr / total_valid_foods
  sym_acc = num_sym / total_valid_foods

  return acc, sym_acc

In [219]:
import pandas as pd

# CSV 파일 읽기
df = pd.read_csv('C:/Users/woobi/Documents/LearningFoodPreferences2/data/wweia_food_categories_addtl.csv')

# 식품대분류 및 상품번호 모으기
category_no_lists = df.groupby('식품대분류')['NO'].apply(list)

# 출력을 위한 딕셔너리 생성
output_dict = {}

# 해당 형식에 맞게 딕셔너리 채우기
for i, (category, no_list) in enumerate(category_no_lists.items(), 1):
    output = []
    no_list.sort()
    start_no = no_list[0]
    end_no = no_list[0]
    for no in no_list[1:]:
        if no == end_no + 1:
            end_no = no
        else:
            output.append(f"{start_no - 1}:{end_no - 1}")
            start_no = no
            end_no = no
    output.append(f"{start_no - 1}:{end_no - 1}")
    output_dict[f'"{category}"'] = output

# 원하는 형식으로 출력
for key, value in output_dict.items():
    print(f"{key} : {value}")


"곡류 및 서류" : ['49:62', '5374:5375', '6759:6767', '6771:6773', '7345:7345']
"과자류" : ['265:269', '1185:1185', '1187:1199', '1398:1402', '1621:1622', '2170:2173', '2276:2276', '2283:2283', '2285:2285', '2300:2305', '2308:2309', '2312:2316', '2346:2348', '2358:2359', '2378:2378', '2380:2380', '2583:2588', '2595:2600', '3200:3200', '3766:3775', '3777:3783', '3786:3791', '3793:3794', '4047:4047', '4059:4062', '4066:4066', '4070:4070', '4091:4091', '4095:4096', '4394:4400', '4402:4404', '4673:4673', '4785:4785', '4795:4796', '5047:5047', '5049:5049', '5093:5093', '5095:5095', '5098:5098', '5105:5105', '5107:5107', '5115:5115', '5169:5169', '5172:5174', '5178:5178', '5186:5188', '5190:5192', '5194:5207', '5209:5213', '5256:5256', '5535:5537', '5539:5539', '6143:6145', '6228:6228', '6270:6271', '6278:6279', '6302:6302', '6359:6364', '6475:6487', '6493:6502', '6506:6508', '6511:6512', '6522:6524', '6598:6598', '6614:6615', '6728:6733', '6736:6736', '6758:6758']
"구이류" : ['0:14', '453:453', '771:77

In [221]:
def get_food_prefs(food_log_categories, wweia_cat_nums_to_words):

  wweia_food_cats_list = wweia_food_categories['NO'].tolist()

  # Make dict to hold the number of foods that belong to a
  # specific WWEIA category. Key is the broad food category.
  cats_dict = {}
  cats_dict["국 및 탕류"] = defaultdict(int)
  cats_dict["튀김류"] = defaultdict(int)
  cats_dict["볶음류"] = defaultdict(int)
  cats_dict["회류"] = defaultdict(int)
  cats_dict["구이류"] = defaultdict(int)
  cats_dict["기타"] = defaultdict(int)
  
  # Dict with lists containing cat codes for each broad food category
  category_mapping = { "국 및 탕류" : wweia_food_cats_list[15:34],
  "튀김류" : wweia_food_cats_list[38:38],
  "볶음류" : wweia_food_cats_list[35:36], 
  "회류" : wweia_food_cats_list[39:40], 
  "구이류" : wweia_food_cats_list[0:14],
  "기타": wweia_food_cats_list[37:37], 
  "곡류 및 서류": wweia_food_cats_list[49:62] + wweia_food_cats_list[5374:5375] + wweia_food_cats_list[6759:6767] + wweia_food_cats_list[6771:6773] + wweia_food_cats_list[7345:7345]}

  broad_categories_list = ["국 및 탕류", "튀김류", "볶음류", "회류", "구이류", "기타"]

  for category_code in food_log_categories:

    if math.isnan(category_code): continue # food does not have valid category, i.e. antacid

    # Loop through each category code and add 1 to its dict entry
    # if it's the right one
    for broad_cat in broad_categories_list:
      if category_code in category_mapping[broad_cat]:
        cats_dict[broad_cat][category_code] += 1
        continue

  max_code_dict = {}
  
  for broad_cat in broad_categories_list:

    if len(list(cats_dict[broad_cat].keys())) > 0:
      max_code_dict[broad_cat] = max(cats_dict[broad_cat], key=cats_dict[broad_cat].get)

    else:
      max_code_dict[broad_cat] = "None of this category was eaten"

  return max_code_dict

In [194]:
def make_food_prefs_table(true_food_prefs, pred_food_prefs, wweia_synonym_cats):
  
  acc = 0
  sym_acc = 0

  for key in true_food_prefs:
    if true_food_prefs[key] == pred_food_prefs[key]:
      acc += 1
      sym_acc += 1
    else:
      if true_food_prefs[key] not in wweia_synonym_cats: continue
      elif pred_food_prefs[key] in wweia_synonym_cats[true_food_prefs[key]]:
        sym_acc += 1

  acc = acc / len(list(true_food_prefs.keys()))
  sym_acc = sym_acc / len(list(true_food_prefs.keys()))

  return acc, sym_acc

In [202]:
def get_pref_metrics(curr_log, n):
  
  true_pop_cats = get_most_pop_categories(curr_log, n, 'wweia_food_category_code')
  pref_pop_cats = get_most_pop_categories(curr_log, n, 'predicted_categories_number')

  true_pop_cats_sym = set()
  pref_pop_cats_sym = set()

  for elem in true_pop_cats:
    if elem in wweia_synonym_cats:
      for sym_elem in wweia_synonym_cats[elem]:
        true_pop_cats_sym.add(sym_elem)

  for elem in pref_pop_cats:
    if elem in wweia_synonym_cats:
      for sym_elem in wweia_synonym_cats[elem]:
        pref_pop_cats_sym.add(sym_elem)

  cats_common = set(true_pop_cats).intersection(set(pref_pop_cats))
  percent_common = len(cats_common) / n

  cats_common_sym = set(true_pop_cats_sym).intersection(set(pref_pop_cats_sym))
  percent_common_sym = min(1, len(cats_common_sym) / n)

  return percent_common, percent_common_sym

In [222]:
# Main method

avg_acc = 0.0
avg_sym_acc = 0.0
avg_pref_acc = 0.0
avg_pref_sym_acc = 0.0
avg_percent_common = 0.0
avg_percent_common_sym = 0.0
avg_mean_rr = 0.0
avg_sym_mean_rr = 0.0

food_log_names = []
accs = []
sym_accs = []
pref_accs = []
pref_sym_accs = []
percent_common_list = []
percent_common_sym_list = []
mean_rrs = []
sym_mean_rrs = []
PATH_NAME2 = 'C:/Users/woobi/Documents/LearningFoodPreferences2/'
path = PATH_NAME2 + "sample_food_logs"

for food_log_name in os.listdir(path):
  
  curr_log = pd.read_csv(os.path.join(path, food_log_name))

  with open(PATH_NAME + 'wweia_synonym_cats.pickle', 'rb') as handle:
    wweia_synonym_cats = pickle.load(handle)
  
  curr_log = process_food_log(curr_log, wweia_synonym_cats)

  food_log_names.append(food_log_name)

  # # Write to file saved in Google Drive folder
  # file_name = PATH_NAME + 'lfp_method4/' + food_log_name
  # curr_log.to_csv(file_name)

  # Compare to true food categories
  acc, sym_acc = get_metrics(curr_log, wweia_synonym_cats)
  avg_acc += acc
  avg_sym_acc += sym_acc
  accs.append(acc)
  sym_accs.append(sym_acc)
  print("Accuracy", acc, "Average Synonymous Accuracy:", sym_acc)

  # Count most popular categories and get food preferences
  with open(PATH_NAME + 'wweia_cat_nums_to_words.pickle', 'rb') as handle:
    wweia_cat_nums_to_words = pickle.load(handle)

  percent_common, percent_common_sym = get_pref_metrics(curr_log, 10)
  print("Percent Top N Categories Identified:", percent_common, "Percent Top N Synonymous Categories Identified:", percent_common_sym)
  avg_percent_common += percent_common
  avg_percent_common_sym += percent_common_sym
  percent_common_list.append(percent_common)
  percent_common_sym_list.append(percent_common_sym)

  true_food_prefs = get_food_prefs(curr_log.loc[:,'wweia_food_category_code'].to_list(), wweia_cat_nums_to_words)
  pred_food_prefs = get_food_prefs(curr_log.loc[:,'predicted_categories_number'].to_list(), wweia_cat_nums_to_words)

  print(true_food_prefs)
  print(pred_food_prefs)

  pref_acc, pref_sym_acc = make_food_prefs_table(true_food_prefs, pred_food_prefs, wweia_synonym_cats)
  avg_pref_acc += pref_acc
  avg_pref_sym_acc += pref_sym_acc
  pref_accs.append(pref_acc)
  pref_sym_accs.append(pref_sym_acc)
  print("Preference Accuracy:", pref_acc, "Preference Synonymous Accuracy:", pref_sym_acc)

  mean_rr = np.mean(curr_log.loc[:,'reciprocal_rank'])
  sym_mean_rr = np.mean(curr_log.loc[:,'sym_reciprocal_rank'])
  avg_mean_rr += mean_rr
  avg_sym_mean_rr += sym_mean_rr
  mean_rrs.append(mean_rr)
  sym_mean_rrs.append(sym_mean_rr)
  print("Mean Reciprocal Rank:", mean_rr, "Synonymous Mean Reciprocal Rank:", sym_mean_rr)


# Averaged statistics for all food logs
# avg_acc /= len(os.listdir(path))
# avg_sym_acc /= len(os.listdir(path))
# avg_pref_acc /= len(os.listdir(path))
# avg_pref_sym_acc /= len(os.listdir(path))
# avg_percent_common /= len(os.listdir(path))
# avg_percent_common_sym /= len(os.listdir(path))
# avg_mean_rr /= len(os.listdir(path))
# avg_sym_mean_rr /= len(os.listdir(path))
# print("Average Accuracy:", avg_acc, "Average Synonymous Accuracy:", avg_sym_acc)
# print("Average Mean Reciprocal Rank:", avg_mean_rr, "Average Synonymous Mean Reciprocal Rank:", avg_sym_mean_rr)
# print("Average Preference Accuracy:", avg_pref_acc, "Average Synonymous Preference Accuracy:", avg_pref_sym_acc)
# print("Average Percent Top N Categories Identified:", avg_percent_common, "Average Percent Top N Synonymous Categories Identified:", avg_percent_common_sym)




['꿩불고기']
7682
['양념왕갈비']
7682
['곰치국']


C:\Users\woobi\AppData\Local\Temp\ipykernel_28752\2266546795.py:93: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0. 0. 0. 0. 0.]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  curr_log.loc[i, 'max_cosim_score'] = np.array2string(to_keep[0,-5:])


7682
['떡만둣국']
7682
['만둣국']
7682
['맑은감자국']
7682
['매생이국']
7682
['무된장국']
7682
['라조기']
7682
['물회']
7682
Accuracy 0.0 Average Synonymous Accuracy: 1.0
Percent Top N Categories Identified: 0.0 Percent Top N Synonymous Categories Identified: 0.0
{'국 및 탕류': 17, '튀김류': 'None of this category was eaten', '볶음류': 'None of this category was eaten', '회류': 40, '구이류': 1, '기타': 'None of this category was eaten'}
{'국 및 탕류': 'None of this category was eaten', '튀김류': 'None of this category was eaten', '볶음류': 'None of this category was eaten', '회류': 'None of this category was eaten', '구이류': 'None of this category was eaten', '기타': 'None of this category was eaten'}
Preference Accuracy: 0.5 Preference Synonymous Accuracy: 0.5
Mean Reciprocal Rank: 0.00036409289184948266 Synonymous Mean Reciprocal Rank: 1.0


In [205]:
# Make table to show accuracies
df = pd.DataFrame(list(zip(food_log_names, accs, sym_accs, pref_accs, pref_sym_accs, percent_common_list, percent_common_sym_list, mean_rrs, sym_mean_rrs)),
               columns =['Food Log', 'Accuracies', "Synomynous Accuracies", "Preference Accuracies", "Synomynous Preference Accuracies", "Percent of Top N Categories Achieved", "Percent of Top N Categories Achieved Sym", "Mean RR", "Sym Mean RR"])
df

,Food Log,Accuracies,Synomynous Accuracies,Preference Accuracies,Synomynous Preference Accuracies,Percent of Top N Categories Achieved,Percent of Top N Categories Achieved Sym,Mean RR,Sym Mean RR
0,sample_food_log.csv,0.0,1.0,0.4,0.4,0.0,0.0,0.000364,1.0
